In [1]:
import time
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [ ]:
import time
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

CHROMEDRIVER_PATH = "C:\\Users\\miche\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=chrome_options)


BASE_URL = "https://www.oiq.qc.ca/en/futurs-membres/devenir-ingenieur-au-quebec".rstrip('/')
visited = set()
all_pages = []

def crawl(url):
    if url in visited:
        return
    visited.add(url)

    try:
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        page = {
            "url": url,
            "title": soup.title.string.strip() if soup.title else "",
            "text": soup.get_text(separator="\n", strip=True)
        }
        all_pages.append(page)

        for link in soup.find_all("a", href=True):
            href = link["href"]
            full_url = urljoin(url, href).split("#")[0].rstrip("/")
            if full_url.startswith(BASE_URL) and full_url not in visited:
                crawl(full_url)
                time.sleep(0.5)

    except Exception as e:
        print(f"❌ Failed to fetch {url}: {e}")


crawl(BASE_URL)
driver.quit()

print(f"\n✅ Scraped {len(all_pages)} pages.")

with open("oiq_scraped_data.json", "w", encoding="utf-8") as f:
    json.dump(all_pages, f, ensure_ascii=False, indent=2)

print("📁 Data saved as oiq_scraped_data.json")


✅ Scraped 9 pages.
📁 Data saved as oiq_scraped_data.json
